In [6]:
import arviz as az
import numpy as np
from tqdm.notebook import  tqdm
from IPython.display import display as print
# import importlib
# importlib.reload(core)
import glob
import pickle
import os

from core_lib import MyDataset, plot_dataset, r2_loss,get_predict_ks


In [7]:

res_path_list = glob.glob("saved_idata-v2/*")
dataset = MyDataset("dataset/data.csv")
df = dataset.get_df()
cct_names, rates_names, error_names = dataset.get_var_col_names()


title_str = "k10,k_kinetics"

for c_name in cct_names:
    title_str += f",{c_name}_r2"
    
title_str += ",r2_mean,r2_all"

print(title_str)

'k10,k_kinetics,xNH3_r2,xNO3_r2,xNO2_r2,xNOrg_r2,xN2_r2,ANH3_r2,ANO3_r2,ANO2_r2,ANOrg_r2,AN2_r2,r2_mean,r2_all'

In [8]:
res_path_list = glob.glob("saved_idata-v2/*")
dataset = MyDataset("dataset/data.csv")
df = dataset.get_df()
cct_names, rates_names, error_names = dataset.get_var_col_names()
c0 = df[cct_names].values[0]
t_eval = df['time'].values

print(c0)
print(t_eval)

array([1.16666667e+00, 2.39880952e+00, 3.11000000e-02, 6.22850000e+01,
       7.11000000e-05, 7.94863498e+00, 6.85792330e-02, 7.23100000e-02,
       9.85666700e-03, 8.32000000e-03])

array([  0.5,  48. ,  96. , 144. ])

In [ ]:
res_path_list = glob.glob("saved_idata-v2/*")

dataset_path = "dataset/data.csv"
dataset = MyDataset(dataset_path)
df = dataset.get_df()
cct_names, rates_names, error_names = dataset.get_var_col_names()
c0 = df[cct_names].values[0]
t_eval = df['time'].values


res_csv = open("res_r2-v2.csv", 'w+', encoding="utf8")
title_str = "k10,k_kinetics"
for c_name in cct_names:
    title_str += f",{c_name}_r2"

title_str += ",r2_mean,r2_all"
res_csv.write(f"{title_str}\n")


# res_path_list = res_path_list[:2]
for index, res_file_path in enumerate(res_path_list):
    
    print(f"{index+1}/{len(res_path_list)} {res_file_path}")
    
    line_str = ""
    
    k_10 = os.path.basename(res_file_path).split('-')[0]
    k_10_2 = bin(int(k_10))[2:]
    k_kinetics = f"{k_10_2:0>11}"
    
    line_str += f"{k_10},{k_kinetics}"
    
    f_size = os.path.getsize(res_file_path)
    if f_size<1:
        continue
    
    idata = pickle.load(open(res_file_path, 'rb'))
    predict_ks = get_predict_ks(idata)

    ks, k_kinetics = predict_ks, k_kinetics

    _db2 = MyDataset(dataset_path=dataset_path)
    _db2.set_as_sim_dataset(t_eval, c0, args=(predict_ks, k_kinetics))
    c_df = _db2.get_df()

    # core.plot_dataset(dataset, _db2)

    r2_losses = []
    for cct_name in cct_names:
        _l = r2_loss(df[cct_name], c_df[cct_name])
        line_str += f",{_l}"
        r2_losses.append(_l)
    r2_all = r2_loss(df[cct_names].values, c_df[cct_names].values)
    r2_mean = np.array(r2_losses).mean()
    line_str += f",{r2_mean}, {r2_all}"
    res_csv.write(f"{line_str}\n")

res_csv.close()